In [ ]:

!pip install -q git+https://github.com/openai/whisper.git
!pip install -q vosk jiwer
!apt-get install -y ffmpeg


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from google.colab import files

uploaded = files.upload()

# pick first uploaded wav/mp3 file
AUDIO_FILE = [f for f in uploaded.keys() if f.endswith(('.wav','.mp3'))][0]
print("Using audio file:", AUDIO_FILE)


Saving new_audio.wav to new_audio.wav
Using audio file: new_audio.wav


In [ ]:
import whisper

# load Whisper model
model = whisper.load_model("base")

# transcribe audio
whisper_result = model.transcribe(AUDIO_FILE)

# show transcription
print("Whisper Transcript:\n")
print(whisper_result["text"])

# save transcript
with open("whisper_transcript.txt", "w") as f:
    f.write(whisper_result["text"])


100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 36.6MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Whisper Transcript:

 I wonder how many meetings is talking about the stuff with the meetings. Yeah, yeah, yeah, yeah. Look at all these stuff, right? Yeah. Okay. Okay. So, what do we need to talk about? Should we just go around and everyone says what they've been doing? Is it not about anything? Not at all. Why, I'm sorry. Yeah. Okay. Sounds like you've done something. That's not it. Well, I've got a browser now, which, whoops, all right, go. Good stuff. Mm-hmm. Well, I dropped mine as well. Good at this. This was something good started. Okay. So, yeah, we've got a browser, which comes up automatically. Which comes up automatically with the transcription box and the topics. And then when you go on the menu, you can select the summarisation box, which pops up, and an audio player. Right. And I think the search works as well. So, pop up a search. It loads up just the background window, so empty. And so, when you start, you have to either open a particular observation or do a search and 

In [ ]:
import os, wave, json
from vosk import Model, KaldiRecognizer

# download a small English model if not already present
if not os.path.exists("vosk-model-small-en-us-0.15"):
    !wget -q https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
    !unzip -q vosk-model-small-en-us-0.15.zip

# load model
vosk_model = Model("vosk-model-small-en-us-0.15")

# open audio file
wf = wave.open(AUDIO_FILE, "rb")
rec = KaldiRecognizer(vosk_model, wf.getframerate())
rec.SetWords(True)

# run recognition
vosk_text = ""
while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        result = json.loads(rec.Result())
        vosk_text += " " + result.get("text", "")
final_result = json.loads(rec.FinalResult())
vosk_text += " " + final_result.get("text", "")

# show transcript
print("Vosk Transcript:\n")
print(vosk_text)

# save transcript
with open("vosk_transcript.txt", "w") as f:
    f.write(vosk_text)


Vosk Transcript:

 when winter comes to the meetings as talking about the stuff of the meetings yeah nevertheless the here get certificate right so we really need to talk about the bodies are we just go around and everyone says what they what they've been doing that without anything live say yeah sounds like you've done i could do that i've got a browser now which whoops no good stuff when of mine as well mississippi given this this wasn't a good start okay sorry yeah say yeah we've got browser read which comes up automatically with or transcription box and the topics and then when you go on the menu you can select some ization box which pops up and an audio player one am and as i think search works as well say pop up a search and loads up just the background when nice empty end so when you start you have to either open open up seek the observation or do such an open it through that that make sense em the transcription boss has got a summarize book button which doesn't do anything it i

In [ ]:
from jiwer import wer

whisper_text = whisper_result["text"].strip()
vosk_text = vosk_text.strip()

# use Whisper transcript as reference, compare Vosk
wer_whisper = wer(whisper_text, whisper_text)  # always 0
wer_vosk = wer(whisper_text, vosk_t9ext)

print("Whisper WER:", round(wer_whisper, 2))
print("Vosk WER   :", round(wer_vosk, 2))


Whisper WER: 0.0
Vosk WER   : 0.66


In [ ]:
files.download("whisper_transcript.txt")
files.download("vosk_transcript.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>